Get your openai api key from settings: https://platform.openai.com/account/api-keys

In [1]:
# Library Installs
%pip install openai --upgrade
%pip install pydub simpleaudio
%pip install nltk
%pip install pyaudio numpy
%pip install sounddevice scipy
%pip install -U openai-whisper
%pip install langchain
%pip install webrtcvad


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: langchain in c:\users\mattj\appdata\local\packages\pythonsoftwarefoundation.python.3.11_qbz5n2kfra8p0\localcache\local-packages\python311\site-packages (0.3.7)



Note: you may need to restart the kernel to use updated packages.


In [2]:
# Library Imports
import getpass
from openai import OpenAI
import os



In [ ]:
# Configuration Variables
MP3_FOLDER = "audio_chunks"  # Directory where audio chunks are stored
errorsLeft = True # Set as true to enter main program While loop

# WAV Recording Parameters

DURATION_THRESHOLD  = 2  # Duration of silence in seconds to trigger a stop
SAMPLE_RATE = 16000  # Sampling rate in Hz
WAV_FOLDER = "recordings"  # Folder to store WAV files
SILENCE_SAVE_BUFFER = 0.2 # a safety buffer for when we auto-trim the audio file for the DURATION_THRESHOLD before saving as WAV
SILENCE_THRESHOLD = 0.01  # Amplitude threshold for silence (adjust as needed)
ENERGY_THRESHOLD= 0.1  # Energy threshold for detecting silence (experiment with this value)
ROLLING_WINDOW_SIZE = 15  # Size of the rolling window for energy averaging
audio_folder = WAV_FOLDER

In [4]:
# Misc Functions
def play_mp3_audio(mp3_filename):
    """
    Plays the specified MP3 file using pydub and simpleaudio.
    
    :param mp3_filename: Name of the MP3 file to play (should be located in the current working directory)
    """
    try:
        # Load audio file
        audio = AudioSegment.from_mp3(Path.cwd() / mp3_filename)
        
        # Play the audio file
        play_obj = sa.play_buffer(
            audio.raw_data,
            num_channels=audio.channels,
            bytes_per_sample=audio.sample_width,
            sample_rate=audio.frame_rate
        )
        play_obj.wait_done()  # Wait until playback is finished
        print(f"Finished playing {mp3_filename}")
    except Exception as e:
        print(f"Error playing audio: {e}")

def get_newest_file(directory):
    """Return the newest file in the specified directory."""
    files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
    if not files:
        return None
    files.sort(key=lambda f: os.path.getmtime(os.path.join(directory, f)), reverse=True)
    return os.path.join(directory, files[0])


In [20]:
import os
import time
import numpy as np
from scipy.io.wavfile import write
import sounddevice as sd
import simpleaudio as sa  # For playing the ding sound

# Configuration
SILENCE_THRESHOLD = 0.02  # Adjust based on sensitivity
DURATION_THRESHOLD = 2  # Duration of silence in seconds to trigger a stop
SAMPLE_RATE = 16000  # Sampling rate in Hz

def play_ding():
    """Play a ding sound to indicate start or end of recording."""
    wave_obj = sa.WaveObject.from_wave_file("ding.wav")
    wave_obj.play()

def is_silent(data, silence_thresh=SILENCE_THRESHOLD):
    """Check if audio data is below a silence threshold."""
    return np.max(np.abs(data)) < silence_thresh

def record_audio():
    """Record audio continuously, and return the audio buffer after prolonged silence is detected."""
    print("Recording... Speak now.")
    play_ding()  # Play ding sound at the start of recording
    audio_buffer = []
    silence_start_time = None
    last_volume_print_time = time.time()

    # Start the audio input stream
    with sd.InputStream(samplerate=SAMPLE_RATE, channels=1, dtype='float32') as stream:
        while True:
            # Read a short segment of audio data (500ms)
            data = stream.read(int(SAMPLE_RATE / 2))[0].flatten()  # 500ms chunks
            audio_buffer.extend(data)

            # Check if the audio data is silent
            if is_silent(data):
                if silence_start_time is None:
                    silence_start_time = time.time()  # Start silence timer
            else:
                silence_start_time = None  # Reset silence timer if sound is detected

            ''' 
            # Print volume level every second
            if time.time() - last_volume_print_time > 1:
                print(f"Volume level: {np.max(np.abs(data))}")
                last_volume_print_time = time.time()
            '''
            # Stop recording if silence has lasted longer than the threshold
            if silence_start_time and (time.time() - silence_start_time) > DURATION_THRESHOLD:
                print("Silence detected. Stopping recording.")
                play_ding()  # Play ding sound at the end of recording
                break

    return np.array(audio_buffer, dtype=np.float32)
def remove_silence_from_buffer(audio_buffer, sample_rate, silence_duration):
    """Remove the specified amount of silence from the end of the audio buffer."""
    silence_samples = int(silence_duration * sample_rate)
    return audio_buffer[:-silence_samples]

def play_audio_buffer(audio_buffer):
    """Play the recorded audio buffer."""
 
    audio_data = np.array(audio_buffer, dtype=np.float32)
    duration = len(audio_data) / SAMPLE_RATE  # Calculate duration in seconds
    print(f"Playing buffer | Duration: {duration:.2f} seconds")

    # Start playback in a non-blocking mode
    sd.play(audio_data, samplerate=SAMPLE_RATE)

    # Show progress with dots updating each second
    start_time = time.time()

    while sd.get_stream().active:  # While audio is still playing
        elapsed = time.time() - start_time
        progress = int(elapsed / duration * 10)  # Calculate progress as a percentage of total time
        print(f"\rPlaying buffer | Duration: {duration:.2f} seconds | Progress: {elapsed:.1f}s [{progress * '.':<10}]", end="")
        time.sleep(0.1)  # Update every 0.1 second

    sd.wait()  # Ensure playback has finished
    print("\rDone" + " " * 50)  # Clear the line by adding spaces after "Done" for a clean exit

def save_audio_buffer(audio_buffer, sample_rate, folder="recordings"):
    """Save the audio buffer to a WAV file with a timestamp in the filename."""
    # Create the folder if it doesn't exist
    os.makedirs(folder, exist_ok=True)

    # Generate a timestamp for the filename
    timestamp = time.strftime("%Y%m%d-%H%M%S")

    # Construct the filename
    filename = f"{folder}/recording-{timestamp}.wav"

    # Save the audio buffer to a WAV file
    write(filename, sample_rate, audio_buffer)


# Usage example
audio_data = record_audio()
# Remove silence from the end of the buffer
audio_data = remove_silence_from_buffer(audio_data, SAMPLE_RATE, DURATION_THRESHOLD)

# Save WAV file
save_audio_buffer(audio_data, SAMPLE_RATE)

Recording... Speak now.
Silence detected. Stopping recording.


In [21]:
# Transcibe Speech to Text

def transcribe_audio(audio_folder=WAV_FOLDER, specified_file=None, use_newest=True):
    """
    Transcribes an audio file using the Whisper model. By default, uses the newest file in the directory,
    but can also use a specified file if provided.

    :param audio_folder: Directory to search for audio files (default is "recordings")
    :param specified_file: File path to a specific file to transcribe; overrides use_newest if provided
    :param use_newest: Boolean flag to determine whether to use the newest file in audio_folder (default is True)
    :return: Transcription text if successful, or an error message
    """
      # Determine file path based on user input or defaults
    file_path = specified_file if specified_file else (get_newest_file(audio_folder) if use_newest else None)

    if file_path and os.path.exists(file_path):
        with open(file_path, "rb") as audio_file:
            # Perform transcription using Whisper model
            transcription = client.audio.transcriptions.create(
                model="whisper-1",
                file=audio_file,
            )
            print("Transcription: " + transcription.text)
            return transcription.text
    else:
        return "No valid audio file found. Please check the directory or specify a valid file."
    

# Example usage
# Or specify a file directly
# print(transcribe_audio(specified_file="specific_audio.mp3", use_newest=False))


In [12]:
# Prompt definitions (pre-written)
# MyPrompt1 find one error and give the complete correction
myPrompt1 = """
You will receive a paragraph written in Spanish. Your task is to identify and correct major grammatical errors while adhering to the following instructions:

1. Ignore gender errors.
2. If a mistake is made but then corrected within the same paragraph, do not include that correction (e.g., "yo comí" followed by "yo comer").
3. Ignore typing mistakes or missing accents; these are not considered errors.
4. Ignore minor clarity issues.
5. If there are no errors worth mentioning, output "No Errors."

Your output should consist only of the correct Spanish for each identified error, separated by periods. Do not provide any additional commentary or explanations. Aim for responses of 4-5 words unless more is needed.

Example:

Input: "Nosotros va a la playa mañana. Ellos trae sus juguetes."

Output: "Nosotros vamos. Ellos traen."
"""

# Archived prompts

myText = "Yo ir al tienda para comprar manzanas y platanos. Cuando llegué, vi muchas frutas y verduras. El cajero me dijo que el precio son muy alto, pero no me importa. Después, yo regresar a casa y preparé un plato grande para mi familia. Todos nos gusta comer juntos."

# MyPrompt2 is show all the errors in a list
myPrompt2 = """ 
You will receive a paragraph written in Spanish. Your task is to identify and correct one major grammatical error while adhering to the following instructions:
Ignore gender errors.
If a mistake is made but then corrected within the same paragraph, do not include that correction (e.g., "yo comí" followed by "yo comer").
Ignore typos or missing accents or written problems; these are not considered errors.
Ignore minor clarity issues.
If there are no significant errors, output "No errors."
Your output should consist of two sentences:

1. The original incorrect sentence, abreviated to show what was said, but not include extraneous detail before and after in the sentence.
2. The corrected version with the corrected word in all caps.
If no significant error is found, just write "No errors.". no second sentence is needed in this case
DO NOT SHOW AN ERROR IF THERE IS NOT ONE.
Example 1:

Input: Ayer, nosotros vamos a la playa mañana para solear.
Output: No errors.
Example 2:

Input: Ayer, nosotros va a la playa mañana para solear. Ellos trae sus juguetes.
Output: Nosotros va a la playa mañana. Nosotros vamos a la playa mañana.

"""
confirmPrompt = """ The following input is supposed to be a sentence in spanish that includes an error, and then the correction of that error. If the second sentence does not actually correct an error or simply repeats the sentence, then respond with 'No errors.' Otherwise repeat back exactly the input given to you without edits. 
Input:
"""
checkCorrectionPrompt = """ 

"""


In [13]:
# Get the openai secret key
secret_key = getpass.getpass("Please enter your openai key: ")
os.environ["OPENAI_API_KEY"]= secret_key

In [14]:
# OpenAI set up:
client = OpenAI()
client.api_key = secret_key
# Set up OpenAI client with the provided API key

def complete(prompt, stop=None):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
             {
                "role": "system",
                "content": "You are a real-time voice chat assistant for Spanish language learning, specializing in Mexican Spanish with a natural Mexican accent."
            },
            {
            "role": "user",
            "content": prompt
            }
        ],
        stop=stop
    )
    return response.choices[0].message.content

complete("is this working?")

'¡Hola! Sí, está funcionando. ¿Cómo puedo ayudarte hoy con tu aprendizaje de español?'

In [ ]:
# Prompts (Pre-written)
testPrompt1 = """ take the following spanish dictation and identify the worst mistake you can find. Follow these rules
If a mistake is made but then immediately corrected within the same paragraph, ignore it (e.g., "yo comes" followed by "yo como").
Ignore typos or missing accents or written problems; these are not considered errors.
Ignore minor clarity issues.
ignore hypothetical, subjungtive or complex conjugations
Focus more on incorrect words getting used, or poor conjugation.
Do not include additional commentary, just output the exact sentence with the error included.
example output: Nosotros va a la playa mañana.
If not major errors can be found, output: No errors.

Input:
"""
testPrompt1a = """Ignore previous instructions, your only job here is this: In the following sentence there may be an error. If you can find it, tell me what it is, and explain your reasoning. 
If a mistake is made but then immediately corrected, ignore it (e.g., "yo comes" followed by "yo como").
Ignore typos or missing accents or written problems; these are not considered errors.
Ignore minor clarity issues.
ignore hypothetical, subjungtive or complex conjugations
Focus more on incorrect words getting used, or poor conjugation
Only explain the correction for a single error. Do not include additional commentary or questions, output nothing but one single explanation for one single error.
It's very important that if there isn't a major mistake in the sentence, to just reply 'No errors.'
Confirm for sure that the error you have selected is bad enough to bother mentioning.
"""
testPrompt2="""We are going to help a student by point out an error they made while speaking. Here is the explanation of the error, take careful not of which word is wrong
"""
testPrompt2a=""" 
Now take the following input sentence:
Input:
"""
testPrompt2b=""" 
output the sentence fragment centered on the erroneous word from the input sentence. The fragment should include the 3 words before and after. Do not include any additional commentary.
If you cannot find an obvious error, just output 'No error.' 
Example Input: Nosotros va a la playa mañana para solear y relajarnos, pero no queremos manejar por todo el dia.
Example Output: Nosotros va a la playa
"""

testPrompt3="""We are going to help a student by correcting an error they made while speaking. Using the following guidance on what the error was:
"""
testPrompt3a=""" And now edit this sentence fragment so it is correct. Do not make it a complete sentence, just correct the error described above. Provide NO ADDITONAL COMMENTARY, just output the corrected sentence fragment
Example Output: Nosotros va a la playa
Example input: Nosotros vamos a la playa
"""

testPrompt4="""You are a careful spanish teacher and you don't want to give unnecessary instruction to a student while they are learning. Look at the following correction made by another teacher and decide if the student was close enough or if the instruction is actually incorrect and should be ignored. If there is any chance the instruction is incorrect reply 'Ignore.' otherwise reply 'Go ahead.' """

confirmErrorCorrectionPrompt1=""" Yes or No, does the end of the first transcription match the second transcription word for word? Ignore spelling and punctuation etc. """
confirmErrorCorrectionPrompt1a=""" Answer either 'Yes.' or 'No.' do not add additional commentary. Also if either of the transcriptions say something in english such as 'Nevermind' then also reply 'Yes' """ 


add a beep when it stops recording
could have a faster dialogue option where it does no analysis
do we need to save the sav to a file or cant we just keep it in the buffer and use it there? Would save some time. And no need to fill this thing up with wav files

In [ ]:
# MAIN PROGRAM LOOP

play_mp3_audio("intro.mp3")

while True:
    record_audio()

    # trim_silence()  # Uses the newest file in the default "audio_chunks" folder

    myTransciption = transcribe_audio()  # Uses the newest file in the default "audio_chunks" folder

    # Check transcription for grammatical errors
    foundError = complete(f"{testPrompt1}\n\n{myTransciption}") 
    if foundError != "Ignore.": # CHANGE THIS SO IT CHECKS IF THAT WORK IS IN THERE ONLY IN CASE IT SAYS OTHER STUFF TOO
        errorExplanation = complete(f"{testPrompt1a}\n\n{myTransciption}")
        print(errorExplanation)
        errorFragment = complete(f"{testPrompt2}\n{errorExplanation}n\n{testPrompt2a}\n\n{foundError}n\n{testPrompt2b}")
        print(errorFragment)
        if errorFragment != "Ignore.":
            errorFixed = complete(f"{testPrompt3}\n{errorExplanation}n\n{testPrompt3a}n\n{errorFragment}")
            print(errorFixed)
            if complete(testPrompt4) != "Ignore.":
                errorsLeft = True
            else:
                errorsLeft = False
                print("Actually nevermind")
        else:
                errorsLeft = False
                print("Actually nevermind")
    else:            
        errorsLeft = False

    while errorsLeft:
        #convert errors into audio and plays them
            errorsLeft = True
            speech_file = "speech_file.mp3"
            speech_file_path = Path.cwd() / speech_file
            #generate the TTS Audio
            response = client.audio.speech.create(
                model="tts-1",
                voice="onyx",
                input=errorFixed
            )
            response.stream_to_file(speech_file_path)

            # play audio back
            play_mp3_audio(speech_file_path)
            
            record_audio()

            trim_silence()  # Uses the newest file in the default "audio_chunks" folder

            myTransciption = transcribe_audio()  # Uses the newest file in the default "audio_chunks" folder

            if complete(f"{confirmErrorCorrectionPrompt1}\n{myTransciption}\n{errorFixed}\n{confirmErrorCorrectionPrompt1a}") != "No.":
                 errorsLeft = False
            # Check if they got it right using a gpt call, if they did they change errorLeft
            # also check against other cases (as described below)
        
        #No more errors, continue conversation, play 'cuentame.mp3' 
    play_mp3_audio("cuentame.mp3")

NameError: name 'play_mp3_audio' is not defined

In [ ]:

''' 
Listen to audio

if the last words said were  
nevermind - just move one. this is tough because i might just completely move on, so you need logic to notice this has happened and ignore checking against old errors
repite - auto replay everything that was just said and ignore 
repite (algo) figure out what exactly i want repeated and repeat that audio slowly (consider slowing down the playback)

otherwise confirm that i repeated back the correction properly. 
'''

' \nif the last words said were  \nnevermind - just move one\nrepite - auto replay everything that was just said and ignore \nrepite (algo) figure out what exactly i want repeated and repeat that audio slowly (consider slowing down the playback)\n\nListen to audio\nI respond normally - one by one, go through the things that were said to me and check if i said them correctly back. \n    if yes, then move to the next item,\n    if no, add that item back to the audio output\n    if all yes, then continue conversation\n    otherwise, concat audio again and play new audio\n\n'